In [ ]:
#Only run in colab
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Github/Taskmaster
#!pip install tensorflow --upgrade
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

In [1]:
import small_rubiks as rubiks
import numpy as np
from student import student
from classroom import classroom
from teacher import teacher
from small_rubiks_neural_networks import student_network
from utils import dotdict
from keras import regularizers

task = rubiks.rubiks_task
setup = rubiks.rubiks_setup
rng = np.random.default_rng(seed=0)

params = dotdict({
            "residual_weights_reg" : regularizers.l2(0.001),
            "residual_bias_reg" : regularizers.l2(0.01),
            "relu_leak" : 0.1,
            "residual_units" : 200,
            "learning_rate" : 0.001,
            "residual_layers" : 3
})
my_student_network = student_network.create(params)
#my_student_network = student_network.load_value_network("models/trained.h5")
student_template = student(task, 50,my_student_network)
t = teacher(setup, lambda n : 1*np.ones(n,dtype=int))##1+rng.poisson(lam=3,size=n)
c = classroom(task, setup, t, student_template, n_students=1, max_steps=10, buffer_size = lambda n : 2048)


In [2]:
c.max_steps=5
t.step_dist = lambda n : 3*np.ones(n,dtype=int)

In [3]:

for i in range(10):
    print(f"Round {i}.")
    c.run_training_batch(n_problems=100,epochs_per_episode=3)
    #if i % 10 == 9:
    #    my_student_network.save("models/trained.h5")

Round 0.
Before step 1, 100 out of 100 remain open.


AttributeError: 'numpy.ndarray' object has no attribute 'depth'

In [5]:
from small_rubiks_neural_networks import student_network
import numpy as np
import small_rubiks as rubiks
from utils import dotdict
from keras import regularizers

params = dotdict({
    "residual_weights_reg" : regularizers.l2(0),
    "residual_bias_reg" : regularizers.l2(0),
    "relu_leak" : 0.1,
    "residual_units" : 100,
    "learning_rate" : 0.001,
    "residual_layers" : 4
})

my_student_network = student_network()
my_student_network.build_state_network(params)

n=4096
rng = np.random.default_rng(seed=0)
states = np.arange(24)[None,:].repeat(n,axis=0)

for k in range(15):
    for i in range(30):
        _, states = rubiks.task_action(states,rng.choice(a=12,size=n))

    actions = rng.choice(12,size=n)
    _, next_states = rubiks.task_action(states,actions)
    my_student_network.fit_state(states,actions,next_states,epochs = 10)

Epoch 1/10
120/120 [==============================] - 3s 6ms/step - loss: 0.6253 - val_loss: 0.4650
Epoch 2/10
120/120 [==============================] - 0s 4ms/step - loss: 0.4461 - val_loss: 0.4431
Epoch 3/10
120/120 [==============================] - 0s 4ms/step - loss: 0.4259 - val_loss: 0.4346
Epoch 4/10
120/120 [==============================] - 0s 3ms/step - loss: 0.4116 - val_loss: 0.4288
Epoch 5/10
120/120 [==============================] - 0s 4ms/step - loss: 0.4003 - val_loss: 0.4252
Epoch 6/10
120/120 [==============================] - 0s 4ms/step - loss: 0.3905 - val_loss: 0.4197
Epoch 7/10
120/120 [==============================] - 0s 4ms/step - loss: 0.3819 - val_loss: 0.4166
Epoch 8/10
120/120 [==============================] - 0s 4ms/step - loss: 0.3740 - val_loss: 0.4122
Epoch 9/10
120/120 [==============================] - 0s 4ms/step - loss: 0.3671 - val_loss: 0.4092
Epoch 10/10
120/120 [==============================] - 0s 3ms/step - loss: 0.3602 - val_loss: 0.4064

In [ ]:
states = np.arange(24)[None,:]
actions = np.array([0])
prediction = my_student_network.predict_state(states,actions)
prediction.reshape(1,24,6), prediction.reshape(1,24,6).argmax(axis=2), rubiks.start_coloring[rubiks.task_action(states,actions)[1][0]]